# DAT600


### Ardijan Rexhaj, Daniel Alvsåker, Ove Oftedal


## Task 1:

### a\) 1.

1 -> [2] <br>
2 -> [2, 3, 4] <br>
3 -> [1, 2, 5] <br>
4 -> [5, 6] <br>
5 -> [3, 4] <br>
6 -> [4] <br>


### a\) 2.

```
     1◄───┐
     │    │
     ▼    │
  ┌─►2◄──►3
  └──┤    ▲
     │    │
     ▼    ▼
     4◄──►5
     ▲
     │
     ▼
     6
```

### a\) 3.

A -> [B]
B -> [C,D]
C -> [E,F]
D -> [E,F]
E -> [F,J]
F -> [B,J,G,H]
G -> []
H -> [I]
I -> []
J -> [I]



### b\)

Breadth first search:

| ![Image 1](./task_b/breadth_first/1.png) | ![Image 2](./task_b/breadth_first/2.png) | ![Image 3](./task_b/breadth_first/3.png) |
|:---:|:---:|:---:|
| ![Image 4](./task_b/breadth_first/4.png) | ![Image 5](./task_b/breadth_first/5.png) | ![Image 6](./task_b/breadth_first/6.png) |
| ![Image 7](./task_b/breadth_first/7.png) | ![Image 8](./task_b/breadth_first/8.png) | ![Image 9](./task_b/breadth_first/9.png) |
| ![Image 10](./task_b/breadth_first/10.png) | ![Image 11](./task_b/breadth_first/11.png) |  |


Depth first search:

![""](./task_b/depth_first/20.png)


### c\)

In order to make the graph on figure 1 into a directed acyclic graph, we can remove the (F,B) edge.


![""](./task_b/topologically_sorted.png)

The topological sort produces the following dependency sequence:
```
[A,B,D,C,E,F,J,H,I,G]
```



### d\)

An algorithm which converts a directed graph into an directed acyclic graph can be done through the following code. This algorithm works by first detecting a cycle through the recursion stack, which simply keeps track of how we recursed depth first to see if any edges point back to a node on that stack, if it does then we remove the edge and check the other nodes.


In [7]:
use std::collections::HashMap;

fn print_graph(graph: HashMap<usize, Vec<usize>>){
    for i in 0..graph.len(){
        println!("vertex: {}, connections: {:?}", i , graph.get(&i).unwrap())
    }
}

fn add_edge(a: usize, b: usize, graph: &mut HashMap<usize, Vec<usize>>) {
    graph.entry(b).or_default();
    graph.entry(a).and_modify(|x| x.push(b)).or_insert(vec![b]);
}

fn find_and_remove_cycles_recursive(
    node: usize,
    visited: &mut Vec<bool>,
    recursion_stack: &mut Vec<bool>,
    graph: &mut HashMap<usize, Vec<usize>>,
) -> bool {
    if recursion_stack[node] {
        return true;
    }

    if visited[node] {
        return false;
    }

    visited[node] = true;

    recursion_stack[node] = true;

    let connected_nodes = graph.get(&node).unwrap().clone();

    for cn in connected_nodes {
        if find_and_remove_cycles_recursive(cn, visited, recursion_stack, graph) {
            // if a connected node returns true here it means the edge
            // is producing a cycle we can thus remove the connection
            graph.entry(node).and_modify(|x| x.retain(|y| *y != cn));
        }
    }

    recursion_stack[node] = false;

    false
}

fn find_and_remove_cycles(graph: &mut HashMap<usize, Vec<usize>>){
    let mut visited = vec![false; graph.len()];
    let mut recursion_stack = vec![false; graph.len()];

    for i in 0..graph.len() {
        find_and_remove_cycles_recursive(i, &mut visited, &mut recursion_stack, graph);
    }
}



let mut graph: HashMap<usize, Vec<usize>> = HashMap::new();

add_edge(0, 1, &mut graph);
add_edge(0, 2, &mut graph);
add_edge(1, 2, &mut graph);
add_edge(2, 0, &mut graph); // Cycle
add_edge(2, 1, &mut graph); // Cycle

find_and_remove_cycles(&mut graph);

println!("Graph Test");
print_graph(graph);

let mut graph: HashMap<usize, Vec<usize>> = HashMap::new();

//Figure 1 but letters are replaced with numbers
add_edge(0, 1, &mut graph);
add_edge(1, 2, &mut graph);
add_edge(1, 3, &mut graph);
add_edge(2, 4, &mut graph);
add_edge(2, 5, &mut graph);
add_edge(3, 4, &mut graph);
add_edge(3, 5, &mut graph);
add_edge(4, 5, &mut graph);
add_edge(4, 6, &mut graph);
add_edge(4, 9, &mut graph);
add_edge(5, 6, &mut graph);
add_edge(5, 7, &mut graph);
add_edge(5, 9, &mut graph);
add_edge(5, 1, &mut graph); // Cycle, F->B
add_edge(7, 8, &mut graph);
add_edge(9, 8, &mut graph);
add_edge(8, 2, &mut graph);
add_edge(8, 2, &mut graph); // Cycle, I->C
add_edge(2, 0, &mut graph); // Cycle, C->A

find_and_remove_cycles(&mut graph);
println!("Graph from fig 1");
print_graph(graph);





Graph Test
vertex: 0, connections: [1, 2]
vertex: 1, connections: [2]
vertex: 2, connections: []
Graph from fig 1
vertex: 0, connections: [1]
vertex: 1, connections: [2, 3]
vertex: 2, connections: [4, 5]
vertex: 3, connections: [4, 5]
vertex: 4, connections: [5, 6, 9]
vertex: 5, connections: [6, 7, 9]
vertex: 6, connections: []
vertex: 7, connections: [8]
vertex: 8, connections: []
vertex: 9, connections: [8]


## Task 2:

### a\)

In this case it is possible to connect all the edges with a cost of just 26, this algorithm centers around making a locally optimal decision based on a starting point which in this case was the cheapest edge. From there we continually expand the network with the cheapest node possible until all nodes are connected.

In [8]:
const A: usize = 0;
const B: usize = 1;
const C: usize = 2;
const D: usize = 3;
const E: usize = 4;
const F: usize = 5;
const G: usize = 6;
const H: usize = 7;


fn find_cheapest_network(mut edges: Vec<(usize, usize, u32)>, vertecies: Vec<usize>) {
    // sort them by cost
    edges.sort_by_key(|&(_, _, cost)| cost);

    let cheapest_edge = edges[0];
    let mut connected_vertecies = vec![cheapest_edge.0, cheapest_edge.1];
    let mut network = vec![cheapest_edge];

    while connected_vertecies.len() != vertecies.len() {
        let mut filtered_edges: Vec<(usize, usize, u32)> = edges
            .iter()
            .filter(|&&(u, v, _)| {
                (connected_vertecies.contains(&u) || connected_vertecies.contains(&v)) // edges must have a vertex in the network
                        && !(connected_vertecies.contains(&u) && connected_vertecies.contains(&v)) // both vertecies cannot be in the network
                        // && ( // Limit the number of edges vertex D can have.
                        //     network.iter().filter(|&&(x, y,_)| (u==D || v==D) && x == D || y == D ).count()
                        // )<3
            })
            .cloned()
            .collect::<Vec<_>>();
        filtered_edges.sort_by_key(|&(_, _, cost)| cost);

        let cheapest_connection_to_new_vertex = filtered_edges[0];
        network.push(cheapest_connection_to_new_vertex);

        if !connected_vertecies.contains(&cheapest_connection_to_new_vertex.0) {
            connected_vertecies.push(cheapest_connection_to_new_vertex.0)
        }
        if !connected_vertecies.contains(&cheapest_connection_to_new_vertex.1) {
            connected_vertecies.push(cheapest_connection_to_new_vertex.1)
        }
    }

    let cost: u32 = network.iter().map(|&(_, _, cost)| cost).sum();

    println!("Network: {:?}", network);
    println!("Cost: {:?}", cost);
    println!("Connected Vertices: {:?}", connected_vertecies);
}


let edges = vec![
    (A, B, 5),
    (A, D, 1),
    (B, D, 4),
    (B, H, 8),
    (C, D, 2),
    (C, G, 6),
    (D, E, 2),
    (D, F, 4),
    (E, H, 8),
    (F, G, 9),
    (F, H, 7),
];

let vertices = vec![A, B, C, D, E, F, G, H];

find_cheapest_network(edges, vertices);

Network: [(0, 3, 1), (2, 3, 2), (3, 4, 2), (1, 3, 4), (3, 5, 4), (2, 6, 6), (5, 7, 7)]
Cost: 26
Connected Vertices: [0, 3, 2, 4, 1, 5, 6, 7]


### b\)


If we restrict the vertex D to only have up to 3 nodes, we cannot achieve a cost below or equal to 30.

The code is the same as above, but now we add a condition to limit the number of edges for node D, it then becomes impossible to achieve the cost limit of 30.


Extra condition, commented out in the code block above:
```
&& (
    network.iter().filter(|&&(x, y,_)| (u==D || v==D) && x == D || y == D ).count()
)<3
```

Output from running the code with condition:
```

Network: [(0, 3, 1), (2, 3, 2), (3, 4, 2), (0, 1, 5), (2, 6, 6), (1, 7, 8), (5, 7, 7)]
Cost: 31
Connected Vertices: [0, 3, 2, 4, 1, 6, 7, 5]

```

It is not possible to achieve a cost cap of 30, the reason for this is that the algorithm knows of the entire network and the cost to each connection, choosing the cheapest edge to connect a new node will always produce the optimal solution. A situation where this might not work is if the algorithm was not aware of all possible edges.

### c\)


Currently the cost is 26 without the limitation of the number of edges that D can have and swapping the cost of edges will certainly produce a network that is below the original cost.

Generally it is possible to improve the network cost iff:

There exists an edge which we do not use, which is cheaper than our most expensive edge.


In our case an optimal swap would be, (5,7,7) which is the edge E-H with a cost of 7. This can be swapped for the cheapest edge which we do not use, which has a cost of 5 between A-B, thus achieving a cost of 24.

Changes made to edges:

```
    let edges = vec![
        // (A, B, 5),
        (A, B, 7),
        (A, D, 1),
        (B, D, 4),
        (B, H, 8),
        (C, D, 2),
        (C, G, 6),
        (D, E, 2),
        (D, F, 4),
        (E, H, 8),
        (F, G, 9),
        (F, H, 5),
        // (F, H, 7),
    ];
```

Output
```
Network: [(0, 3, 1), (2, 3, 2), (3, 4, 2), (1, 3, 4), (3, 5, 4), (5, 7, 5), (2, 6, 6)]
Cost: 24
Connected Vertices: [0, 3, 2, 4, 1, 5, 7, 6]
```